# MVP Análise de Dados e Boas Práticas

Nome: Daniel Waissmann Szyfman

Matrícula: 4052024001726

Dataset: [Car Evaluation Dataset](https://archive.ics.uci.edu/dataset/19/car+evaluation)

# Introdução

A avaliação de veículos é uma etapa essencial para a tomada de decisão por parte de consumidores, fabricantes e concessionárias. O *Car Evaluation Dataset*, disponibilizado pela UCI Machine Learning Repository, oferece uma base estruturada para análise automatizada de atributos de veículos, permitindo classificar o quão aceitável é um carro com base em critérios técnicos e financeiros.

Este projeto tem como objetivo aplicar técnicas de análise exploratória e pré-processamento de dados para investigar padrões dentro deste dataset e preparar os dados para futuros modelos de machine learning. Utilizaremos a linguagem Python e bibliotecas como `pandas`, `numpy` e `seaborn`, sempre buscando adotar boas práticas de organização de código e narrativa interpretativa.

Ao longo deste notebook, buscarei construir um storytelling claro: desde a definição do problema até o tratamento dos dados. A intenção é não apenas preparar os dados tecnicamente, mas também gerar insights práticos sobre os critérios que influenciam na avaliação de um veículo, e como esses critérios se relacionam entre si.

# Definição do Problema

O objetivo deste trabalho é compreender os fatores que influenciam a avaliação de veículos com base em seus atributos técnicos e econômicos. O conjunto de dados utilizado traz características como preço de compra, número de portas, capacidade de carga e segurança, entre outras variáveis categóricas.

## Tipo de Problema

Estou lidando com um problema de **classificação supervisionada**, no qual cada instância representa um veículo com características específicas, e a variável de saída indica o nível de aceitabilidade desse carro. A variável alvo possui quatro possíveis categorias: `unacc`, `acc`, `good` e `vgood`.

O desafio é identificar como os atributos fornecidos contribuem para essa avaliação final e compreender os padrões que diferenciam carros mal avaliados daqueles considerados bons ou muito bons.


## Hipóteses

Antes de iniciar a análise detalhada, levantei algumas hipóteses que pretendo verificar ao longo do trabalho:

- Carros com maior capacidade de carga e mais portas tendem a ser melhor avaliados.
- Veículos com manutenção muito cara ou baixa segurança provavelmente são classificados negativamente.
- Pode haver combinações específicas entre segurança e preço de compra que afetam diretamente a avaliação, mesmo quando os demais atributos são favoráveis.
- A segurança pode ter maior peso na classificação do que o número de passageiros ou o tamanho do porta-malas.
- Veículos baratos com boa segurança podem ter avaliações boas mesmo com menor conforto.

Essas hipóteses servirão como base para guiar a análise exploratória e a interpretação dos dados.


## Seleção de Dados

O dataset escolhido foi o *Car Evaluation Dataset*, originalmente publicado no UCI Machine Learning Repository. A base contém apenas atributos categóricos, que representam diferentes aspectos técnicos e funcionais de um veículo, e é amplamente utilizada para tarefas de classificação.

A escolha foi motivada pela simplicidade estrutural, facilidade de entendimento e riqueza de combinações possíveis entre os atributos, o que permite uma análise interpretativa eficaz e um fluxo de pré-processamento direto.


## Atributos do Dataset

A base de dados é composta pelos seguintes atributos:

- **buying**: custo de compra (valores: vhigh, high, med, low)
- **maint**: custo de manutenção (valores: vhigh, high, med, low)
- **doors**: número de portas (valores: 2, 3, 4, 5more)
- **persons**: capacidade de pessoas (valores: 2, 4, more)
- **lug_boot**: tamanho do porta-malas (valores: small, med, big)
- **safety**: nível de segurança (valores: low, med, high)

A variável alvo (que representa a classificação final do veículo) é:

- **class**: avaliação final do carro (valores: unacc, acc, good, vgood)

Todos os atributos do conjunto de dados são **categóricos com ordenação natural**. Isso significa que, apesar de representados por strings, existe uma hierarquia implícita entre os valores de cada atributo. Essa característica será essencial no momento do pré-processamento, quando precisarei aplicar uma codificação que preserve essa ordem.



# Importação das Bibliotecas Necessárias e Carga de Dados

Nesta etapa, realizarei a importação das bibliotecas essenciais para análise, visualização e preparação dos dados. Também farei o carregamento inicial do dataset para dar início à exploração.

In [1]:
# Bibliotecas para manipulação e análise de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações opcionais para os gráficos
# sns.set(style="whitegrid")
# plt.rcParams["figure.figsize"] = (10, 6)
